In [4]:
from venv import create
import psycopg2
from psycopg2.extras import execute_values
from pgvector.psycopg2 import register_vector
from psycopg2 import sql

import pandas as pd
import numpy as np

In [3]:
def connect_cursor():
    '''Connects to postgres, returns (connection, cursor)'''
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="receipts",
            user="postgres",
            password='postgres'
        )
        cur = conn.cursor()
        # Register the vector type with psycopg2
        register_vector(conn)
    except:
        print('Error connecting to database')
    return (conn, cur)

In [5]:
# Setup
def setup_vector():
    try:
        conn = psycopg2.connect(
            host="localhost",
            database="receipts",
            user="postgres",
            password='postgres'
        )
        cur = conn.cursor()
        
        # Install pgvector
        cur.execute("CREATE EXTENSION IF NOT EXISTS vector");
    except:
        print('Error connecting to database')
    conn.commit()
    conn.close()

In [11]:
def setup_rewe_table():
    '''Fills rewe table with store products, embeddings'''
    conn, cur = connect_cursor()

    df_rewe = pd.read_csv('data/name_embeds_incl_special_items_no_context.csv', index_col=0)

    data_list = [(row['name'], row['price'], row['category'], row['embeddings']) for _, row in df_rewe.iterrows()]
    #print(data_list[0:5])
    #print(df_rewe)
    
    try:
        execute_values(cur, "INSERT INTO rewe (name, price, category, embedding) VALUES %s", data_list)
    except Exception as e:
        print(f"Error occurred: {e}")
    conn.commit()
    conn.close()

In [12]:
'''Run setup
    
Installs vector extension to postgres
Sets up receipts table
Sets up rewe table
Fill rewe table with products and embeddings'''

# Install pgvector
setup_vector()

# Fill in rewe table
setup_rewe_table()